In [21]:
#!pip install dbdpy
#!pip install dbfread
#!pip install pandas
#!pip install bar_chart_race
#!pip install matplotlib pandas


In [16]:
import os
import pandas as pd
from dbfread import DBF

In [40]:
#Especificamos la ruta de los archivos DFB
CARPETA_PALTA = "D:/Proyectos/Ranking Palta/ranking-palta/Datos"

#creamos una lista en donde se van a almacenar los dataframes
dataframes = []

for archivo in os.listdir(CARPETA_PALTA):
    if archivo.endswith(".DBF"):
        # Leemos el archivo
        rutaArchivo = os.path.join(CARPETA_PALTA, archivo)        
        tabla = DBF(rutaArchivo, encoding="latin1")
        df = pd.DataFrame(iter(tabla))
        dataframes.append(df)

# Concatenamos todos los Dataframes en uno solo
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Convertimos la columna 'FECHA' a texto
df_concatenado['FECHA'] = df_concatenado['FECHA'].astype(str)

# Creamos la lista de posiciones que queremos cambiar
posiciones = [4, 6]

# Creamos una función para insertar una misma letra en posiciones específicas
def insertar_letras(valor, letra, posiciones):
    for posicion in sorted(posiciones, reverse=True):
        valor = valor[:posicion] + letra + valor[posicion:]
    return valor

# Aplicamos la función a la columna del dataframe
df_concatenado['FECHA_MOD'] = df_concatenado['FECHA'].apply(lambda x: insertar_letras(x, "/", posiciones))
df_concatenado['FECHA_MOD_FER'] = pd.to_datetime(df_concatenado["FECHA_MOD"], format = "%Y/%m/%d")

# Creamos una tabla dinámica de los datos que queremos
tabla_dinamica= pd.pivot_table(
    df_concatenado,
    values=["FOB_DOLPOL"],
    index= "FECHA_MOD_FER",
    columns= "NRO_DOCU",
    aggfunc= {
        "FOB_DOLPOL": "sum"
    }

)

df_tabla_dinamica=tabla_dinamica.reset_index()
print(tabla_dinamica)

               FOB_DOLPOL                                                  \
NRO_DOCU      20100007852 20100032709 20100039207 20100085225 20100699801   
FECHA_MOD_FER                                                               
2001-01-29            NaN         NaN         NaN         NaN         NaN   
2001-03-21            NaN         NaN         NaN         NaN         NaN   
2001-04-01            1.0         NaN         NaN         NaN         NaN   
2001-04-29            NaN         NaN         NaN         NaN         NaN   
2001-05-14            NaN         NaN         NaN         NaN         NaN   
...                   ...         ...         ...         ...         ...   
2024-08-29            NaN         NaN         NaN         NaN         NaN   
2024-08-30            NaN         NaN         NaN         NaN         NaN   
2024-08-31            NaN         NaN         NaN         NaN         NaN   
2024-09-01            NaN         NaN         NaN         NaN         NaN   

In [28]:
#Seleccionamos las columnas que nos interesa
df_top10=df_concatenado[["FECHA_MOD_FER", "NRO_DOCU", "FOB_DOLPOL"]]
print(df_top10.head())

  FECHA_MOD_FER     NRO_DOCU  FOB_DOLPOL
0    2024-04-19  20604580294         128
1    2024-06-29  20103272964       37440
2    2024-06-13  20170040938       29299
3    2024-06-13  20170040938       32820
4    2024-06-21  20103272964       37440


In [19]:
#Análisis para Limpieza de datos
"""#FrecuenciaTexto
frecuencia = df_concatenado["DESC_COM"].value_counts()
#Porcentaje de texto
porcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100

#Creamos un dataframe con los resultados
resultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )
#Calculamos el porcentaje acumulado
resultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()
#Filtramos los que llegan al 80%

resultados= resultados[resultados["Porcentaje acumulado"]>80]
print(resultados)"""

'#FrecuenciaTexto\nfrecuencia = df_concatenado["DESC_COM"].value_counts()\n#Porcentaje de texto\nporcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100\n\n#Creamos un dataframe con los resultados\nresultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )\n#Calculamos el porcentaje acumulado\nresultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()\n#Filtramos los que llegan al 80%\n\nresultados= resultados[resultados["Porcentaje acumulado"]>80]\nprint(resultados)'

In [20]:
df["DESC_COM"].unique

<bound method Series.unique of 0                PALTAS FRESCAS
1      PULPA DE PALTA CONGELADA
2                PALTAS FRESCAS
3           PALTAS REFRIGERADAS
4           PALTAS REFRIGERADAS
                 ...           
99          PALTAS REFRIGERADAS
100         PALTAS REFRIGERADAS
101         PALTAS REFRIGERADAS
102         PALTAS REFRIGERADAS
103              PALTOS FRESCOS
Name: DESC_COM, Length: 104, dtype: object>